<a href="https://colab.research.google.com/github/360abhi/FirebaseAngularApp/blob/main/Zeus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import spacy

import re

# spacy.cli.download("en_core_web_sm")


nlp = spacy.load("en_core_web_sm")

pre_def_verbs = ['click','submit','review','edit','enter','view','check','access','select','authorize']


def find_screen_names(sentence):
    pattern = r"(?i)(?:move to|navigate to|switch to)\s+'?(?:the|a)?\s*([\w\s]+)'?(?:\s+tab|\s+screen|\s+page|\s+window)?|'([\w\s]+)'\s+(?:tab|screen|page|window)|(?:\bin\b)\s+'?([\w]+)\s+(?:tab|screen|page|window)'?"

    matches = re.findall(pattern, sentence)
    screen_names = [name1 or name2 or name3 for name1, name2, name3 in matches if name1 or name2 or name3]

    # Remove duplicates
    screen_names = list(set(screen_names))
    return screen_names

def separate_screen_words(screen_names):
    words = []
    for name in screen_names:
        words.extend(name.split())
        words.extend(['tab','screen','window'])
    return words


def extract_action_verbs(sentence):


    doc = nlp(sentence)

    action_verbs = []

    field =[]

    screens = []
    sep_screens=[]

    data =[]

    screens = find_screen_names(sentence)
    sep_screens = separate_screen_words(screens)
    print(sep_screens)


    rootToken = next(token for token in doc if token.dep_ == "ROOT")


    for index,token in enumerate(doc):

            if token.dep_ == "ROOT":

                # Check if ROOT and does not have xcomp children

                if not any(child.dep_ in ["ccomp","xcomp","pcomp"] for child in token.children):

                  if token.pos_ == "VERB":

                    action_verbs.append(token.text)

                    if token.text.lower() not in pre_def_verbs:

                      pre_def_verbs.append(token.text.lower())

                      # print(token.text.lower())

                  elif token.pos_ == "PROPN" and token.text not in screens:
                    field.append(token.text)



                # If ROOT and has xcomp children, skip ROOT but consider xcomp

                else:

                    for child in token.children:

                        if child.dep_ in ["ccomp","pcomp","xcomp"] and child.pos_ == "VERB":



                            action_verbs.append(child.text)

                            if child.text.lower() not in pre_def_verbs:

                              pre_def_verbs.append(child.text.lower())

                              print(child.text.lower())


            elif token.dep_ in ["xcomp","ccomp","pcomp"]:

              # print("Insidexcomp", token.text)

              try:

                if token.head.text != rootToken.text:

                  if token.pos_ == "VERB":

                    action_verbs.append(token.text)

                    if token.text.lower() not in pre_def_verbs:

                      pre_def_verbs.append(token.text.lower())

                      print(token.text.lower())



                    # for child in token.children:



                    #   if child.dep_ == "dobj":

                    #       field = child.text



              except:

                pass

            elif token.dep_ == "conj":

                # Check if conj has xcomp children and the child is a verb

                if any(child.dep_ in ["xcomp","pcomp","ccomp"] and child.pos_ == "VERB" for child in token.children):

                    for child in token.children:

                        if child.dep_ in ["xcomp","pcomp","ccomp"] and child.pos_ == "VERB":

                            action_verbs.append(child.text)

                            if child.text.lower() not in pre_def_verbs:

                              pre_def_verbs.append(child.text.lower())

                              print(child.text.lower())



                            # for child in token.children:



                            #   if child.dep_ == "dobj":

                            #       field = child.text



                # If conj itself is a verb and has no xcomp children, append conj



                elif token.pos_ == "VERB":

                    action_verbs.append(token.text)

                    if token.text.lower() not in pre_def_verbs:

                      pre_def_verbs.append(token.text.lower())


                    # for child in token.children:



                    #   if child.dep_ == "dobj":

                    #       field = child.text



            if token.dep_ in ["dobj","pobj","PROPN"] and token.text not in sep_screens:

              # field.append(token.text)

              # var_field = token.text

              field.append(token.text)

              prev = doc[index-1]


              if prev.pos_ in ["NOUN","ADJ","PROPN"] and prev.text not in sep_screens:

                # field.append(prev.text)

                # var_field = prev.text + " " + var_field

                field.append(prev.text)

                # print(var_field)

                prev2 = doc[index-2]

                if prev2.pos_ in ["NOUN","ADJ","PROPN"] and prev2.text not in sep_screens:

                  # var_field = prev2.text + " " + var_field

                  field.append(prev2.text)

                  # print(var_field)

              # field.append(var_field)

            # if token.dep_ in ["pobj"] and token.text not in sep_screens and len(field) == 0:

            #   # field.append(token.text)

            #   # var_field = token.text

            #   field.append(token.text)

            #   prev = doc[index-1]


            #   if prev.pos_ in ["NOUN","ADJ","PROPN"] and prev.text not in sep_screens:

            #     # field.append(prev.text)

            #     # var_field = prev.text + " " + var_field

            #     field.append(prev.text)

            #     # print(var_field)

            #     prev2 = doc[index-2]

            #     if prev2.pos_ in ["NOUN","ADJ","PROPN"] and prev2.text not in sep_screens:

            #       # var_field = prev2.text + " " + var_field

            #       field.append(prev2.text)


            # if token.dep_ in ["pobj"] and token.text not in sep_screens and len(field) !=0:

            #   # field.append(token.text)

            #   # var_field = token.text

            #   data.append(token.text)

            #   prev = doc[index-1]


            #   if prev.pos_ in ["NOUN","ADJ","PROPN"] and prev.text not in sep_screens:

            #     # field.append(prev.text)

            #     # var_field = prev.text + " " + var_field

            #     data.append(prev.text)

            #     # print(var_field)

            #     prev2 = doc[index-2]

            #     if prev2.pos_ in ["NOUN","ADJ","PROPN"] and prev2.text not in sep_screens:

            #       # var_field = prev2.text + " " + var_field

            #       data.append(prev2.text)


    # PRD check

    for token in doc:

      if token.text.lower() not in action_verbs and token.text.lower() in pre_def_verbs and token.text not in field:

        action_verbs.append(token.text)

    return action_verbs,field,screens



sentences = [



    # "The operational maker should be able to log in to ARX as Operational Maker and see all products as per entitlement.",



    "In package tab Click on Create Package button",



    # "The user should be able to click on the create package button.",



    "The system should allow the user to enter all details, and after clicking the next button, it should navigate to the 'Products' tab.",



    # "The bank user should be able to select all the products.",



    # "The bank user should be able to select all the charge fields.",



    # "The bank user should be able to select all the billing details.",



    # "The bank user should be able to select the template and report for collections + J54.",



    # "The bank user should be able to select the template and report for payments.",



    # "The bank user should be able to select the template for the common report.",



    # "The checker should be able to log in to CIM.",



    # "After clicking save, he submitted the Form",



    # "User should select the charges as done",

    # "Access and Login into ARX Page",

    # "Review and Submit Package"



]



In [71]:
from IPython.display import HTML

import re


def print_colored_sentence(sentence, words, fields,screens):

    words_set = set(words)

    fields_set = set(fields)

    screen_set = set(screens)

    # data_set = set(data)

    color ="red"

    color_field = "blue"

    colored_sentence = ""



    for word in sentence.split():

        word = re.sub(r'[^\w\s]', '', word)

        if word in words_set:

             colored_sentence += (

                f'<span style="color: {"black"}; background-color: {"yellow"}; font-weight: bold;">{word}</span> '

            )

        elif word in fields_set:

          colored_sentence += (

                f'<span style="color: {"white"}; background-color: {"blue"}; font-weight: bold;">{word}</span> '

            )

        # elif word in data_set:

        #   colored_sentence += (

        #         f'<span style="color: {"white"}; background-color: {"blue"}; font-weight: bold;">{word}</span> '

        #     )

        else:

            colored_sentence += word + " "

    for screen in screens:

      if screen in sentence:

        colored_sentence = colored_sentence.replace(screen, f'<span style="color: {"white"}; background-color: {"green"}; font-weight: bold;">{screen}</span>')



    # for field in fields:

    #   if field in sentence:

    #     colored_sentence = colored_sentence.replace(field, f'<span style="color: {"white"}; background-color: {"blue"}; font-weight: bold;">{field}</span>')



    display(HTML(colored_sentence.strip()))

import string

def remove_punctuation(sentence):
    translator = str.maketrans('', '', string.punctuation)
    return sentence.translate(translator)



for sentence in sentences:
    sentence = remove_punctuation(sentence)

    action_verbs,fields,screens = extract_action_verbs(sentence)

    print_colored_sentence(sentence, action_verbs,fields,screens)

['package', 'tab', 'screen', 'window']


['Products', 'tab', 'tab', 'screen', 'window']
clicking


In [73]:
import pandas as pd
import re

file_path = 'CIM_TestCases.xlsx'
df = pd.read_excel(file_path,"Onboarding Package")

step_summary_column = df['Step Summary']

def split_sentences(text):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return sentences

def contains_number(sentence):
    return bool(re.search(r'\d', sentence))

def is_text_sentence(sentence):
    return not contains_number(sentence) and sentence.strip()

all_sentences = []

for text in step_summary_column:
    if pd.notna(text):
        sentences = split_sentences(text)
        filtered_sentences = [s for s in sentences if is_text_sentence(s)]
        all_sentences.append(filtered_sentences)

for idx, sentences in enumerate(all_sentences[10:50]):
    print(f"TestCase {idx + 1}:")
    for sentence in sentences:
      action_verbs,fields,screen = extract_action_verbs(sentence)
      print_colored_sentence(sentence, action_verbs,fields,screen)
      print()

TestCase 1:
[]



[]



['package', 'tab', 'screen', 'window']



[]



['products', 'tab', 'screen', 'window']


...


[]



TestCase 40:
[]



[]



['package', 'tab', 'screen', 'window']



[]
